#**Data Preparation**

First we need to import all the required packages

#**Implementation**

###**Required Packages**

The required python packages are as follows, (here I mentioned the packages with versions that I have used for the developments)

tensorflow==2.3.1

nltk==3.5

colorama==0.4.3

numpy==1.18.5

scikit_learn==0.23.2

Flask==1.1.2

##**Define Intents**

I will define few simple intents and bunch of messages that corresponds to those intents and also map some responses according to each intent category. I will create a JSON file named “intents.json” including these data as follows.



##**Data Preparation**
First we need to import all the required packages

In [3]:
from tensorflow.python.ops.array_ops import sequence_mask
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Now we load the json file and extract the required data.

In [7]:
with open('intents.json') as file:
  data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
  for pattern in intent['patterns']:
    training_sentences.append(pattern)
    training_labels.append(intent['tag'])
  responses.append(intent['responses'])

  if intent['tag'] not in labels:
    labels.append(intent['tag'])

num_classes = len(labels)

Now we are ready to train our model. Simply we can call the “fit” method with training data and labels.

In [10]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [14]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)

#**Model Training**

Let’s define our Neural Network architecture for the proposed model and for that we use the “Sequential” model class of Keras.

In [17]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [44]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels),epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 10ms/step - loss: 0.4844 - accuracy: 1.0000
Epoch 2/500
2/2 [==============================] - 0s 19ms/step - loss: 0.4817 - accuracy: 1.0000
Epoch 3/500
2/2 [==============================] - 0s 12ms/step - loss: 0.4798 - accuracy: 1.0000
Epoch 4/500
2/2 [==============================] - 0s 9ms/step - loss: 0.4781 - accuracy: 1.0000
Epoch 5/500
2/2 [==============================] - 0s 13ms/step - loss: 0.4769 - accuracy: 1.0000
Epoch 6/500
2/2 [==============================] - 0s 13ms/step - loss: 0.4751 - accuracy: 1.0000
Epoch 7/500
2/2 [==============================] - 0s 15ms/step - loss: 0.4730 - accuracy: 1.0000
Epoch 8/500
2/2 [==============================] - 0s 9ms/step - loss: 0.4708 - accuracy: 1.0000
Epoch 9/500
2/2 [==============================] - 0s 13ms/step - loss: 0.4676 - accuracy: 1.0000
Epoch 10/500
2/2 [==============================] - 0s 13ms/step - loss: 0.4639 - accuracy: 1.0000
Epoch 11/500
2/2 [===

After training, it is better to save all the required files in order to use it at the inference time. So that we save the trained model, fitted tokenizer object and fitted label encoder object.

In [40]:
#to save the training model 
model.save('chat_model')

import pickle

#to save the fitted tokenizer
with open('tokenizer_pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

#to save the fitted labeler
with open('label_encoder.pickle','wb') as ecn_file:
  pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

#**Interface**

Okay!!!! now it’s time to check how our model performs. 😊

We are going to implement a chat function to engage with a real user. When a new user message is received, the chatbot will calculate the similarity between the new text sequence and training data. Considering the confidence scores got for each category, it categorizes the user message to an intent with the highest confidence score.

In [41]:
!pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

    print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
    chat()